In [ ]:
import numpy as np
import pandas as pd 
import datetime as dt 
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm

: 

In [ ]:
years = 15

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 365 * years)


In [ ]:
tickers = ['SPY', 'BND', 'GLD', 'QQQ', 'VTI']

In [ ]:
adj_close_df = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start = startDate, end = endDate, auto_adjust= False)
    adj_close_df[ticker] = data["Adj Close"]
print(adj_close_df)

Calculate the daily log returns and drop na

In [ ]:
log_returns = np.log(adj_close_df / adj_close_df.shift(1))
log_returns = log_returns.dropna()
print(log_returns)

Create a function that calculate Exp return (We do assume that futur returns are expected to be as the past returns --> not the most accurate are reliable)

In [ ]:
def expected_return(weights, log_returns):
    return np.sum(log_returns.mean()*weights)

In [ ]:
def standard_dev(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

Create the Cov_matrix for all the securities

In [ ]:
cov_matrix = log_returns.cov()


create an equally weighted PF

In [ ]:
portfolio_value = 1000000

weights = np.array([1/len(tickers)]*len(tickers))

portfolio_expected_return = expected_return(weights, log_returns)

portfolio_std_dev = standard_dev(weights, cov_matrix)

Creation of a function that gives a random Z-score based on normal distribution

In [ ]:
def random_z_score():
    return np.random.normal(0,1)

Create a function to calculate scenario GainLoss

In [ ]:
days = 5

def scenario_gain_loss(portfolio_value, portfolio_std_dev, z_score, days):
    return portfolio_value * portfolio_expected_return * days + portfolio_value * portfolio_std_dev * z_score * np.sqrt(days)

Run 10000 Simulations

In [ ]:
simulations = 10000
scenarioReturn = []

for i in range (simulations):
    z_score = random_z_score()
    scenarioReturn.append(scenario_gain_loss(portfolio_value, portfolio_std_dev, z_score, days))

In [ ]:
confidence_interval = 0.95
VaR = -np.percentile(scenarioReturn, 100 * (1 - confidence_interval))
print(VaR)

Plot the results

In [ ]:
plt.hist(scenarioReturn, bins = 50, density = True)
plt.xlabel('Scenario Gain/Loss ($)')
plt.ylabel('Frequency')
plt.title(f'Distribution of Portfolio Gain/loss Over {days} Days')
plt.axvline(-VaR, color = 'r', linestyle = 'dashed', linewidth = 2, label = f'VaR at {confidence_interval: .0%} confidence level')
plt.legend
plt.show